In [1]:
import numpy as np 
import pandas as pd 
import os
import cv2
import torch

In [2]:
path = ''

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [3]:
train

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0
...,...,...,...,...,...
1816,Train_1816,0,0,0,1
1817,Train_1817,1,0,0,0
1818,Train_1818,1,0,0,0
1819,Train_1819,0,0,1,0


In [4]:
import torchvision
import random
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
from sklearn.model_selection import train_test_split
"""
train, valid = train_test_split(train, 
                                test_size=0.1,
                                stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']],
                                random_state=50)
"""
train, test2 = train_test_split(train, 
                                test_size=0.15,
                                stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']],
                                random_state=50)
train, valid = train_test_split(train, 
                                test_size=0.15,
                                stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']],
                                random_state=50)

In [6]:
from torch.utils.data import Dataset

class ImageDataset(Dataset):
    
    def __init__(self, df, img_dir='images/', transform=None, is_test=False):
        super().__init__() 
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    
    
    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]             
        img_path = self.img_dir + img_id + '.jpg' 
        image = cv2.imread(img_path)              
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        
        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        if self.is_test:
            return image 
        else:
            
            label = np.argmax(self.df.iloc[idx, 1:5]) 
            return image, label 

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform_train = A.Compose([
    A.Resize(450, 650),      
    A.RandomBrightnessContrast(brightness_limit=0.2, 
                               contrast_limit=0.2, p=0.3),
    A.VerticalFlip(p=0.2),    
    A.HorizontalFlip(p=0.5),  
    A.ShiftScaleRotate(       
        shift_limit=0.1,
        scale_limit=0.2,
        rotate_limit=30, p=0.3),
    A.OneOf([A.Emboss(p=1),   
             A.Sharpen(p=1),
             A.Blur(p=1)], p=0.3),
    A.PiecewiseAffine(p=0.3), 
    A.Normalize(),            
    ToTensorV2()              
])

transform_test = A.Compose([
    A.Resize(450, 650), 
    A.Normalize(),      
    ToTensorV2()        
])

/home/lee.soomi/.conda/envs/work/lib/python3.8/site-packages/albumentations/core/validation.py:45: UserWarning: This augmenter is very slow. Try to use ``ElasticTransformation`` instead, which is at least 10x faster.
  original_init(self, **validated_kwargs)


In [8]:
img_dir = 'images/'

dataset_train = ImageDataset(train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(valid, img_dir=img_dir, transform=transform_test)

In [9]:
# Multi-Processing
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

from torch.utils.data import DataLoader

batch_size = 4

loader_train = DataLoader(dataset_train, batch_size=batch_size, 
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g, num_workers=2)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size, 
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g, num_workers=2)

In [21]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b1', num_classes=4) 

model = model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /home/lee.soomi/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


In [11]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006, weight_decay=0.0001)

In [12]:
# get_cosine_schedule_with_warmup: decreases learning rate on cosine schedule
from transformers import get_cosine_schedule_with_warmup

epochs = 39

# scheduler setting
# optimizer: torch.optim.AdamW
# num_warmup_steps: number of steps to increase the learning rate(lr) gradually to the value we set before
# num_training_steps: total number of steps to training = batches * epochs
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                           num_warmup_steps=len(loader_train)*3,
                                           num_training_steps=len(loader_train)*epochs)

In [23]:
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm

for epoch in range(epochs):
    # == [ train phase ] ==============================================
    model.train()        
    epoch_train_loss = 0 
    
    for images, labels in tqdm(loader_train):
        
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() # reset weights of the model in optimizer
        outputs = model(images) # deliver batch of images to the model and get outputs
        loss = criterion(outputs, labels) # calculate the loss
        
        epoch_train_loss += loss.item() 
        loss.backward() # backpropagation
        optimizer.step() # update weights
        scheduler.step() # update scheduler
    
    print(f'epoch [{epoch+1}/{epochs}] - train loss : {epoch_train_loss/len(loader_train):.4f}')
    
    # == [ validation phase ] ==============================================
    model.eval() # set the model to evaluation mode
    epoch_valid_loss = 0  
    preds_list = []       
    true_onehot_list = [] 
    
    # fix weights
    with torch.no_grad(): 
        
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            preds = torch.softmax(outputs.cpu(), dim=1).numpy() # prediction
            
            true_onehot = torch.eye(4)[labels.cpu()].numpy()  
            
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
    
    print(f'epoch [{epoch+1}/{epochs}] - valid loss : {epoch_valid_loss/len(loader_valid):.4f} / valid ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}')
    torch.save(model, 'train3/' + str(epoch))
    

  0%|          | 0/329 [00:00<?, ?it/s]

epoch [1/39] - train loss : 1.3929
epoch [1/39] - valid loss : 1.3943 / valid ROC AUC : 0.5187


  0%|          | 0/329 [00:00<?, ?it/s]

epoch [2/39] - train loss : 1.3907
epoch [2/39] - valid loss : 1.3957 / valid ROC AUC : 0.5104


  0%|          | 0/329 [00:00<?, ?it/s]

epoch [3/39] - train loss : 1.3933
epoch [3/39] - valid loss : 1.3961 / valid ROC AUC : 0.5139


  0%|          | 0/329 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
# Original Test Data

dataset_test = ImageDataset(test2, img_dir=img_dir, transform=transform_test)
loader_test = DataLoader(dataset_test, batch_size=batch_size, 
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)

# Test-Time Augmentation
dataset_TTA = ImageDataset(test, img_dir=img_dir,
                          transform=transform_train, is_test=True)
loader_TTA = DataLoader(dataset_TTA, batch_size=batch_size,
                       shuffle=False, worker_init_fn=seed_worker,
                       generator=g, num_workers=2)

In [28]:
# Original Test Data

from sklearn.metrics import accuracy_score
model_t = torch.load('train2/5', weights_only=False)
model_t.eval() 

preds_test = np.zeros((len(test), 4)) 

with torch.no_grad(): 
        
    for images, labels in tqdm(loader_test):
        images = images.to(device)
        labels = labels.to(device)
            
        outputs = model_t(images)
        loss = criterion(outputs, labels)
        epoch_valid_loss += loss.item()
            
        preds = torch.softmax(outputs.cpu(), dim=1).numpy() # prediction
            
        true_onehot = torch.eye(4)[labels.cpu()].numpy()  
            
        preds_list.extend(preds)
        true_onehot_list.extend(true_onehot)
    
print(f'epoch [{epoch+1}/{epochs}] - Test loss : {epoch_valid_loss/len(loader_valid):.4f} / Test ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}')
#print("Accuracy : ", str(accuracy_score(true_onehot_list, preds_list)))

  0%|          | 0/69 [00:00<?, ?it/s]

epoch [4/39] - Test loss : 2.1110 / Test ROC AUC : 0.9152
